In [38]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from itertools import product
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which sis why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('pattern_eda.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize, float64, uint
# from tslearn.preprocessing import TimeSeriesScalerMeanVariance
# from tslearn.piecewise import PiecewiseAggregateApproximation
# from tslearn.piecewise import SymbolicAggregateApproximation, OneD_SymbolicAggregateApproximation
from scipy.stats import zscore
from sortedcontainers import SortedList, SortedSet 

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 200)
pd.set_option('display.max_columns', 50)

from common_util import RAW_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, DT_BIZ_DAILY_FREQ, get_custom_biz_freq, get_custom_biz_freq_df, query_df, search_df, chained_filter, benchmark
from common_util import MUTATE_DIR, load_df, load_json, outer_join, left_join, count_nn_df, count_nz_df, count_nn_nz_df, pairwise, cust_count, list_get_dict, get_time_mask
from data.data_api import DataAPI
from raw.common import default_row_masksfile
from recon.viz import *
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

## Load Data

### RAW

In [39]:
VOL_DIR = RAW_DIR +'vol/'
PRICE_DIR = RAW_DIR +'price/'

SPX = load_df(PRICE_DIR +'SPX.parquet')
RUT = load_df(PRICE_DIR +'RUT.parquet')
NDX = load_df(PRICE_DIR +'NDX.parquet')
DJI = load_df(PRICE_DIR +'DJI.parquet')
VIX = load_df(VOL_DIR +'VIX.parquet')
RVX = load_df(VOL_DIR +'RVX.parquet')
VXN = load_df(VOL_DIR +'VXN.parquet')
DVX = load_df(VOL_DIR +'VXD.parquet')

### ROOT

In [ ]:
root_rcs, root_dfs = DataAPI.axe_load(['root', 'root_split_ohlca'])
hrm_rcs, hrm_dfs = DataAPI.axe_load(['hrm', 'hrm'], lazy=False)

In [ ]:
ass = 'sp_500'

## VIEW

In [ ]:
root_dfs[[ass, 'root', 'root_split_ohlca', 'join', 'pba']].loc['2016-03-28 13:00':'2016-03-28 22:00'].dropna()

In [ ]:
hrm_dfs[[ass, 'hrm', 'hrm', 'pba', 'pba']].loc['2016-03-28 13:00':'2016-03-28 22:00'].dropna()